In [ ]:
import numpy as np
from scipy.sparse import csc_array
from scipy.sparse.linalg import splu, spsolve

from src.array import Array
from src.matrix import Matrix

In [ ]:
length = 3
width = 2.0
seed = 0
val_cap = 1.0
time_step = 0.01
cond_ext = 1.0

In [ ]:
array = Array(length=length)
matrix = Matrix(matrix_init=None, array=array, cond_ext=cond_ext, val_cap=val_cap, time_step=time_step)

---

In [ ]:
# def draw_matrix(mat: csc_array | np.ndarray):
#     fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=72)
#     mat = mat.toarray() if isinstance(mat, csc_array) else mat
#     size_row, size_col = mat.shape

#     ax.spy(mat)
#     for idx, entry in enumerate(mat.reshape(-1)):
#         if entry:
#             coord_row, coord_col = idx // size_col, idx % size_col
#             ax.text(coord_col, coord_row, str(int(entry)), c="red", horizontalalignment="center", verticalalignment="center")


#     ax.set_xticks(range(size_col))
#     ax.set_xticklabels(["Top", 3, 4, 5, 6, 7, 8] + list(range(12, 27))) # L=3 specific
#     ax.tick_params(axis='x', which='both', labelbottom=True)

#     ax.set_yticks(range(size_row))
#     ax.tick_params(axis='y', which='both', labelright=True)
#     plt.tight_layout()
#     plt.show()

In [ ]:
# draw_matrix(matrix_discharge.div_cap_inc)

---

In [ ]:
# EDGES ORDER , NOT EDGES_DIV_CAP ORDER !!!!!!!!!!!!!!!!!!!!!!!!!!
volts_cap = np.zeros(array.num_edge, dtype=np.float64)
volts_cap[array.idxs_edge_vertical] = 1

In [ ]:
# np.concatenate((np.zeros(matrix.size_div_comb), volts_cap)) # testing output in ipynb

In [ ]:
print("breaking", array.edges[7])
edge_div_cond_broken = array.edges_div_cond[array.idxs_edge_to_edge_div_cond[7]]
print("equivalent Gdiv edge", edge_div_cond_broken)

idx_node1_new, idx_node2_new = edge_div_cond_broken[0] - array.length + 1, edge_div_cond_broken[1] - array.length_double + 1
matrix.div_cond[idx_node1_new, idx_node1_new] -= 1
matrix.div_cond[idx_node2_new, idx_node2_new] -= 1
matrix.div_cond[idx_node1_new, idx_node2_new] += 1
matrix.div_cond[idx_node2_new, idx_node1_new] += 1

matrix.div_comb[idx_node1_new, idx_node1_new] -= matrix.time_step
matrix.div_comb[idx_node2_new, idx_node2_new] -= matrix.time_step
matrix.div_comb[idx_node1_new, idx_node2_new] += matrix.time_step
matrix.div_comb[idx_node2_new, idx_node1_new] += matrix.time_step

In [ ]:
from scipy.sparse import bmat

volts_node_div = spsolve(
    bmat(
        [[matrix.div_cond, matrix.div_cap_inc.T], [matrix.div_cap_inc, None]],
        format="csc",
    ),
    np.concatenate((np.zeros(matrix.size_div_comb), volts_cap)),
)[:matrix.size_div_comb]

In [ ]:
length_minus_one = array.length - 1
length_double_minus_one = array.length_double - 1

def compute_volts_cap(volts_node_div):
    for idx_edge, (idx_node1, idx_node2) in enumerate(np.array(array.edges_div_cap)[array.idxs_edge_to_edge_div_cap].tolist()):
        idx_node1_new, idx_node2_new = idx_node1 - length_minus_one, idx_node2 - length_double_minus_one
        if idx_node1 < length:
            volts_cap[idx_edge] = volts_node_div[0] - volts_node_div[idx_node2_new]
        else:
            volts_cap[idx_edge] = volts_node_div[idx_node1_new] - volts_node_div[idx_node2_new]

In [ ]:
volts_cap_profile = []
volts_cap_profile.append(volts_cap.copy())

for _ in range(3999):
    volts_node_div = spsolve(matrix.div_comb, matrix.div_cap @ volts_node_div, permc_spec="MMD_AT_PLUS_A")
    compute_volts_cap(volts_node_div)
    volts_cap_profile.append(volts_cap.copy())
    

In [ ]:
yd = volts_cap_profile[-1]

In [ ]:
for idx_edge, (idx_node1, idx_node2) in enumerate(np.array(array.edges_div_cap)[array.idxs_edge_to_edge_div_cap].tolist()):
    print((idx_node1, idx_node2), yd[idx_edge])

In [ ]:
import matplotlib.pyplot as plt
volts_cap_profile_temp = np.array(volts_cap_profile.copy()).transpose() # [edge, time]

fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=72)

for idx_edge, volt_profile in enumerate(volts_cap_profile_temp[array.idxs_edge_vertical]):
    ax.plot(range(1000), volt_profile, lw=idx_edge + 2, zorder=20 - idx_edge) #, c="black", lw=1)
    # ax.text(
    #     2000 + np.random.uniform(-200, 200, 1)[0],
    #     volt_profile[-1] + np.random.uniform(-0.1,  + 0.1, 1)[0], 
    #     np.array(array.edges)[array.idxs_edge_horizontal][idx_edge],
    #     zorder=100
    # )

plt.tight_layout()
plt.show()

In [ ]:
asdf

In [ ]:
# # fully connected; from falstad
# volts_node_div = np.array(
#     [
#         1.5,  # VTOP
#         1,  # V3
#         1,
#         1,
#         0.5,
#         0.5,
#         0.5,
#         0.5,  # V12
#         0.5,
#         0.5,
#         1,
#         1,
#         0,
#         1,
#         0,
#         0,
#         0.5,  # V21
#         0.5,
#         -0.5,
#         0.5,
#         -0.5,
#         -0.5,
#     ]
# )

# idx_edge 7 broken; from falstad
volts_node_div = np.array(
    [
        4/3,  # VTOP
        5/6,  # V3
        2/3,
        5/6,
        0.5,
        2/3,
        0.5,
        1/3,  # V12
        1/3,
        1/3,
        2/3,
        5/6,
        -1/6,
        5/6,
        -1/3,
        -1/6,
        2/3,  # V21
        0.5,
        -0.5,
        0.5,
        -1/3,
        -0.5,
    ]
)

In [ ]:
plt.bar(range(22), asdf[:matrix.size_div_comb] - volts_node_div, width=1)

In [ ]:
plt.bar(range(22), asdf[:matrix.size_div_comb], width=1)

In [ ]:
import numpy as np
np.all((val_cap * (matrix.div_cap_inc.T @ matrix.div_cap_inc)).toarray() == matrix.div_cap)